In [ ]:
import pandas as pd

from source_code.core.hypothesis_evaluation.generate_graphs import *

import sys
import altair as alt
from IPython.display import HTML

In [ ]:
alt.data_transformers.disable_max_rows()
alt.renderers.enable('svg')

HTML('''
<svg height="0" width="0" xmlns="http://www.w3.org/2000/svg" version="1.1">
    <defs>

        <!-- From SVG stripe generator 
        https://www.coffee-break-designs.com/labs/svg_stripe_generator/ -->
        <pattern id="pattern_0" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(0)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#CB37A5" stroke-width="15" />
        </pattern>
        
        <pattern id="pattern_1" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(45)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#1F77B4" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_2" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(135)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#FF7F0E" stroke-width="4" />
        </pattern>

        <pattern id="pattern_3" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(0)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#2CA02C" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_4" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(90)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#D62728" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_5" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(90)">
            <line x1="0" y="0" x2="0" y2="5" stroke="#9467BD" stroke-width="4" />
            <line x1="0" y="0" x2="5" y2="0" stroke="#9467BD" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_6" patternUnits="userSpaceOnUse" width="5" height="5" patternTransform="rotate(45)">
            <line x1="0" y="0" x2="5" y2="0" stroke="#8C564B" stroke-width="4" />
            <line x1="0" y="0" x2="0" y2="5" stroke="#8C564B" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_7" patternUnits="userSpaceOnUse" width="10" height="10" patternTransform="rotate(45)">
            <line x1="0" y="0" x2="0" y2="10" stroke="#316B83" stroke-width="4" />
        </pattern>
        
        <pattern id="pattern_8" patternUnits="userSpaceOnUse" width="10" height="10" patternTransform="rotate(135)">
            <line x1="0" y="0" x2="0" y2="10" stroke="#CE6DBD" stroke-width="4" />
        </pattern>
        
    </defs>
</svg>
''')

In [ ]:
r = 'R8'

df_res_s = []
df_stat_s = []

In [ ]:
#Get results file
for k in range(10):
    print(f'------{k}------')
    df_res = pd.read_csv(f'../experiments/MovieLens/{r}/result_group_descriptions_{r}_{k}.csv')
    df_stat = pd.read_csv(f'../experiments/MovieLens/{r}/result_statistics_{r}_{k}.csv')
    
    df_res = df_res.replace('SUPPORT_⍺','Ψ-Support')
    df_res = df_res.replace('FARSIGHT_⍺','β-Farsighted')
    df_res = df_res.replace('FIXED_⍺','γ-Fixed')
    df_res = df_res.replace('HOPE_⍺','ẟ-Hopeful')
    df_res = df_res.replace('HYBRID_⍺','Ɛ-Hybrid')
    
    df_stat = df_stat.replace('SUPPORT_⍺','Ψ-Support')
    df_stat = df_stat.replace('FARSIGHT_⍺','β-Farsighted')
    df_stat = df_stat.replace('FIXED_⍺','γ-Fixed')
    df_stat = df_stat.replace('HOPE_⍺','ẟ-Hopeful')
    df_stat = df_stat.replace('HYBRID_⍺','Ɛ-Hybrid')
    
    df_res.param = df_res.param.apply( lambda x : str(int(x)) if isinstance(x,float)==True and x > 1 else str(x) )
    df_stat.param = df_stat.param.apply( lambda x : str(int(x)) if isinstance(x,float)==True and x > 1 else str(x) )

    df_res.param = df_res.param.apply(lambda x: 'BN' if x=='fdr_b' else 'BY' if x=='fdr_by' else x)
    df_stat.param = df_stat.param.apply(lambda x: 'BN' if x=='fdr_b' else 'BY' if x=='fdr_by' else x)
    
    df_res['algorithm'] = df_res['algorithm']+'_'+df_res['param']
    df_stat['algorithm'] = df_stat['algorithm']+'_'+df_stat['param']
    
    df_res['pairs'] = df_res['group1'] + ' ' + df_res['group2']
    df_res['coverage_gained'] = (df_res['coverage_gained_1'] + df_res['coverage_gained_2'])/2
    
    df_res = df_res.replace('Ψ-Support_0.1666666666666666','Ψ-Support_0.16666666666666666')
    
    df_res_s.append( df_res )
    df_stat_s.append( df_stat )

In [ ]:
num_hyps = [i/100 for i in df_stat_s[0]['sample'].unique()] #Get samples

isin_best = ['COVER_G_BN','COVER_G_BY','β-Farsighted_0.9','γ-Fixed_500','ẟ-Hopeful_500',\
             'Ɛ-Hybrid_(0.75, 200, 200)','Ψ-Support_0.5','COVER_⍺_500']

FDR - POWER

In [ ]:
fdr = get_fdr_power(df_stat_s, df_res_s, num_hyps, isin_best, threshold=20)
fdr

X : Samples , Y : Coverage

In [ ]:
cov_sample = get_cov_samples(df_stat_s, df_res_s, num_hyps, isin_best, threshold=[50])
cov_sample

X : Samples , Y : Time

In [ ]:
time_sample = get_time_sample(df_stat_s, num_hyps, isin_best, threshold=[100])
time_sample

X : n, Y : Time

In [ ]:
tim_n = get_tim_n(df_stat_s, [1], isin_best)
tim_n

Y : Sum Coverage (Sum p-values), X : Steps 

In [ ]:
cov_p_values_n = get_cov_p_values_n(df_stat_s, df_res_s, [1], isin_best, threshold=20)
cov_p_values_n